In [6]:
from ipywidgets import (FileUpload, HBox, VBox, HTML,
                       Layout, Button, Output, Dropdown,
                       FloatText, Checkbox, Combobox)

import PIL.Image as Image
import io

from string import Template

from urllib.parse import unquote, urlparse

from IPython.display import display, clear_output, Javascript
import base64

import bokeh
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.models import (CustomJS, ColumnDataSource, 
                          FileInput, Div, Row,
                          TextInput)
from bokeh import events
from bokeh.io import push_notebook, output_notebook

import plot_digitizer as pdz
from seeq import spy, sdk
import re

import json

output_notebook(INLINE)
# output_notebook()

Loading BokehJS ...

In [ ]:
ALERTS_ON=False
pushed_curve_sets_and_names = {}

In [7]:
def alert_message(title, body):
    tmplt = Template(
        """
        require(
            ["base/js/dialog"], 
            function(dialog) {
                dialog.modal({
                    title: '$title',
                    body: '$body',
                    buttons: {
                        'Dismiss': {}}
                });
            }
        );
        """
    )
    
    display(
        Javascript(
            tmplt.substitute(title=title, body=body)
        ))

In [1]:
def get_existing_set_names(curve_set_dict):
    try:
        return list(curve_set_dict.keys())
    except:
        return []

def get_existing_curve_names(curve_set_dict, set_name):
    try:
        return list(curve_set_dict[set_name].keys())
    except:
        return []

In [8]:
### load workbook, worksheet etc

# get the url
url = unquote(jupyter_notebook_url)


workbook_id_match = re.search('workbookId=\w+-\w+-\w+-\w+\w+-\w+', url)
if workbook_id_match is None:
    raise TypeError('No workbook ID identified in URL.')

wkb_id_end_pos = workbook_id_match.end()
wkb_id = url[wkb_id_end_pos-36:wkb_id_end_pos]

worksheet_id_match = re.search('worksheetId=\w+-\w+-\w+-\w+\w+-\w+', url)
if worksheet_id_match is None:
    raise TypeError('No worksheet ID identified in URL.')

wks_id_end_pos = worksheet_id_match.end()
wks_id = url[wks_id_end_pos-36:wks_id_end_pos]

workbook = pdz.get_workbook(wkb_id)
worksheet = pdz.get_worksheet(workbook, wks_id)

# TODO FIX XAXIS AND Y AXIS
x_axis_id = worksheet.display_items.iloc[0].ID
y_axis_id = worksheet.display_items.iloc[1].ID

items_api = sdk.ItemsApi(spy.client)
trees_api = sdk.TreesApi(spy.client)
workbooks_api = sdk.WorkbooksApi(spy.client)

asset_name_to_item_id_dict = pdz.get_available_asset_names_to_item_id_dict(worksheet, trees_api)
asset_names = asset_name_to_item_id_dict.keys()

initial_asset = pdz.get_asset(asset_name_to_item_id_dict[list(asset_names)[0]], trees_api)
initial_pltdgtz_prop = pdz.get_pltdgtz_property(initial_asset.id, items_api)
try:
    initial_curve_set_dict = json.loads(initial_pltdgtz_prop.value)
    initial_existing_set_names = get_existing_set_names(initial_curve_set_dict)
except AttributeError:
    initial_curve_set_dict = []
    initial_existing_set_names = []

curve_set_dict = initial_curve_set_dict
existing_set_names = initial_existing_set_names

In [9]:
def _add_plot_point(x, y):
    global s, l
    try:
        ox, oy = list(s.data_source.data['x'].values), list(s.data_source.data['y'].values)
    except AttributeError:
        ox, oy = s.data_source.data['x'], s.data_source.data['y']
    ox.append(x)
    oy.append(y)
    try:
        s.data_source.data = {'x':ox, 'y':oy}
        l.data_source.data = {'x':ox, 'y':oy}
    except NameError:
        s.data_source.data = {'x':ox, 'y':oy}
        
    clear_selection.disabled=False
    approve_axis_point.disabled=False
    
    push_notebook()
    
def _clear_plot_points():
    global s, l
    try:
        s.data_source.data = {'x':[], 'y':[]}
        l.data_source.data = {'x':[], 'y':[]}
    except NameError:
        s.data_source.data = {'x':[], 'y':[]}
    approve_axis_point.disabled=True
    clear_selection.disabled=True
    push_notebook()

In [10]:
def digitizer_fig(imgurl, mode:'str'):
    mode = mode.lower()
    allowed = ['scatter', 'digitize']
    if mode not in allowed:
        raise ValueError('mode must be {}, not {}'.format(allowed, mode))
    
    # create a Figure object
    p = figure(x_range=(0,1), y_range=(0,1), tools="pan,reset,crosshair,box_zoom")

    
    x = []
    y = []
    source = ColumnDataSource(data=dict(x=x, y=y))

    callback = CustomJS(args=dict(source=source), code="""
        Jupyter.notebook.kernel.execute(`_add_plot_point(${cb_obj.x}, ${cb_obj.y})`)
    """)
    
    # plotting
    p.image_url(url=[imgurl], x=0, y=1, w=1, h=1)
    global s, l, x1, x2, y1, y2
    l = p.line(x='x',y='y',source=source,color='blue', line_width=4)
    s = p.scatter(x='x',y='y',source=source,color='blue', size=8.5)
    calibration_size = 25
    x1 = p.scatter(x=[], y=[], color='blue', size=calibration_size, marker='+', line_width=4)
    x2 = p.scatter(x=[], y=[], color='yellow', size=calibration_size, marker='+', line_width=4)
    y1 = p.scatter(x=[], y=[], color='purple', size=calibration_size, marker='+', line_width=4)
    y2 = p.scatter(x=[], y=[], color='green', size=calibration_size, marker='+', line_width=4)
        
    p.js_on_event(events.Tap, callback)
    p.xaxis.visible = False
    p.yaxis.visible = False
    p.sizing_mode = 'scale_width'
    return p

In [11]:
xaxis_calibration = []
yaxis_calibration = []

instructions = ["""
    <div style="border:3px; border-style:solid; border-color:blue; padding: 1em;">
    <h5>Instructions: Upload an image.</h5>
    <p>Use the file upload to upload an image.</p>
    </div>
    """,
    """
    <div style="border:3px; border-style:solid; border-color:blue; padding: 1em;">
    <h5>Instructions: Calibrate the x-axis.</h5>
    <p>Select (click) a point on the x-axis of the plot image and manually enter the x-coordinate below.</p>
    </div>
    """,
    
    """
    <div style="border:3px; border-style:solid; border-color:yellow; padding: 1em;">
    <h5>Instructions: Calibrate the x-axis. (Point 2)</h5>
    <p>Select a second point (far from the first) on the x-axis of the plot image and manually enter the x-coordinate below.</p>
    </div>
    """,
    
    """
    <div style="border:3px; border-style:solid; border-color:purple; padding: 1em;">
    <h5>Instructions: Calibrate the y-axis.</h5>
    <p>Select a point on the y-axis of the plot image and manually enter the y-coordinate below.</p>
    </div>
    """,
    
    """
    <div style="border:3px; border-style:solid; border-color:green; padding: 1em;">
    <h5>Instructions: Calibrate the y-axis.</h5>
    <p>Select a second point (far from the first) on the y-axis of the plot image and manually enter the y-coordinate below.</p>
    </div>
    """,
    
][::-1]

curve_select_instructions = """
    <h1>Axis calibration done.</h1>
    <div style="border:3px; border-style:solid; border-color:orange; padding: 1em;">
    <h5>Instructions: Digitize the curve</h5>
    <p>Select points along the curve you wish to digitize. When done, click "Push To Seeq".</p>
    </div>
"""

# widgets
file_upload = FileUpload(accept='.png', multiple=False)

fig_output = Output()
fig_output.layout = Layout(width='60%')

console_output = Output()

asset_name = Dropdown(    
    options=list(asset_names),
    value=list(asset_names)[0],
    description='Asset:',
    disabled=True,
)

set_name = Combobox(
    value=None,
    placeholder='',
    options=initial_existing_set_names,
    description='Curve Set:',
    disabled=True
)

curve_name = Combobox(
    value=None,
    placeholder='',
    options=[],
    description='Curve Name:',
    disabled=True
)


axis_selection_instructions = HTML(instructions.pop())

axis_point_float = FloatText(
    value=0.0,
    description='x-minimum:',
    disabled=True,
    style={'description_width': 'initial'},
    layout = Layout(width='60%')
)

approve_axis_point = Button(
    description='Ok',
    disabled=True,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Select',
    icon='check', # (FontAwesome names without the `fa-` prefix),
    layout=Layout(width='30%', positioning='right')
)

clear_selection = Button(
    description='Clear Selection',
    disabled=True,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Select',
    layout=Layout(width='30%', positioning='left')
)

axis_select_box = HBox(
    (axis_point_float, approve_axis_point),
    layout=Layout(justify_content='space-between', width='100%')
)

push_to_seeq = Button(
    description='Push to Seeq',
    disabled=True,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Push to seeq',
    icon='check', # (FontAwesome names without the `fa-` prefix),
    layout=Layout(width='50%',)
)

digitize_another_button = Button(
    description='Digitize another Curve',
    disabled=True,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Digitize another curve (using the same axis calibration)',
    #icon='check', # (FontAwesome names without the `fa-` prefix),
    layout=Layout(width='50%',)
)

push_to_seeq_box = VBox(
    ( 
        HBox(
            (push_to_seeq,),
            layout=Layout(justify_content='center', width='100%')
        ),
    ),
    layout=Layout(justify_content='flex-end',min_height='100px')
)

digitize_another_box = VBox(
    ( 
        HBox(
            (digitize_another_button,),
            layout=Layout(justify_content='center', width='100%')
        ),
    ),
    layout=Layout(justify_content='flex-end',min_height='100px')
)
    

user_input = VBox(
    (
        asset_name, 
        set_name, 
        curve_name, 
        axis_selection_instructions,
#         axis_calibration_checkboxes,
        axis_select_box,
        clear_selection,
        push_to_seeq_box,
        digitize_another_box
    ),
    layout=Layout(width='35%')
)

# actions
def on_clear_selection(*args):
    _clear_plot_points()
    clear_selection.disabled=True
    

def on_file_upload(change):
    file_upload.disabled = True
    _bytes = file_upload.value[list(file_upload.value.keys())[-1]]['content']
    img = base64.b64encode(_bytes).decode("utf-8") 
    
    global imgurl, fig
    
    imgurl = 'data:image/png;base64,'+img
    fig = digitizer_fig(imgurl, mode='scatter')
    with fig_output:
        clear_output()
        show(fig, notebook_handle=True)
    axis_selection_instructions.value = instructions.pop()
    asset_name.disabled=False
    set_name.disabled=False
    curve_name.disabled=False
    axis_point_float.disabled=False
    with console_output:
        clear_output()
        print('Image loaded.')
#     approve_axis_point.disabled=False
#     clear_selection.disabled=False
        
axis_calibration_counter = 0    

def on_axis_point_approval(*args):
    global axis_calibration_counter, imgurl, s
    global xaxis_calibration, yaxis_calibration
    
    if (len(s.data_source.data['x']) != 1 or len(s.data_source.data['y']) != 1) and ALERTS_ON:
        alert_message('Incorrect number of points selected', 
                      """There are {} points selected. While calibrating axes, please select one point at a time. Clear selection and try again.""".format(
                          max((len(s.data_source.data['x']), len(s.data_source.data['y'])))
                      )
                     )
        return
        
    axis_calibration_counter += 1
    if axis_calibration_counter == 1:
        xaxis_calibration.append((s.data_source.data['x'][0], axis_point_float.value))
        axis_point_float.description = 'x-maximum:'
        x1.data_source.data = dict(s.data_source.data)
    elif axis_calibration_counter == 2:
        xaxis_calibration.append((s.data_source.data['x'][0], axis_point_float.value))
        axis_point_float.description = 'y-minimum:'
        x2.data_source.data = dict(s.data_source.data)
    elif axis_calibration_counter == 3:
        yaxis_calibration.append((s.data_source.data['y'][0], axis_point_float.value))
        axis_point_float.description = 'y-maximum:'
        y1.data_source.data = dict(s.data_source.data)
    elif axis_calibration_counter == 4:
        yaxis_calibration.append((s.data_source.data['y'][0], axis_point_float.value))
        y2.data_source.data = dict(s.data_source.data)
        
    try:
        axis_selection_instructions.value = instructions.pop()
        axis_point_float.value = 0
#         if axis_calibration_counter == 2:
#             x_axis_checkbox.value = True
#             axis_point_float.description = 'y-coordinate:'
        _clear_plot_points()
    except IndexError:
        if axis_calibration_counter >= 4:
#             y_axis_checkbox.value = True
            axis_selection_instructions.value = curve_select_instructions
            approve_axis_point.disabled = True
            axis_point_float.disabled = True
            push_to_seeq.disabled=False
        _clear_plot_points()
        
def on_digitize_another(*args):
    global workbook, worksheet
    with console_output:
        clear_output()
        print('Reloading workbook...')
    workbook = pdz.get_workbook(wkb_id)
    with console_output:
        clear_output()
        print('Reloading worksheet...')
    worksheet = pdz.get_worksheet(workbook, wks_id)
    with console_output:
        clear_output()
        print('Clearing previous selection...')
    _clear_plot_points()
    push_to_seeq.disabled=False
    digitize_another_button.disabled=True
    with console_output:
        clear_output()
        print('Ready for next curve selection.')
    return
    
        
def on_push_to_seeq(*args):
    if (len(s.data_source.data['x']) < 2 or len(s.data_source.data['y']) < 2) and ALERTS_ON:
        alert_message('No curve selected', 
                      """There are {} points selected. When digitizing, you must select at least 3 points. Clear selection and try again.""".format(
                          min((len(s.data_source.data['x']), len(s.data_source.data['y'])))
                      )
                     )
        return
    
    clear_selection.disabled = True
    approve_axis_point.disabled = True
    
    global curve_selection_points
    
    with console_output:
        clear_output()
        print('Pushing to Seeq...')
    
    with console_output:
        clear_output()
        print('Calibrating selection by axes...')
    
    curve_selection_points = pd.DataFrame(data=dict(x=s.data_source.data['x'], y=s.data_source.data['y']))
    
    # convert from calibration
    x_cal_array = np.array(xaxis_calibration)
    x_diff = np.diff(x_cal_array, axis=0).flatten()
    x_min = np.min(x_cal_array, axis=0)
    curve_selection_points['real_x'] = ((curve_selection_points['x'] - x_min[0])/(x_diff[0]))*(x_diff[1]) + x_min[1]

    y_cal_array = np.array(yaxis_calibration)
    y_diff = np.diff(y_cal_array, axis=0).flatten()
    y_min = np.min(y_cal_array, axis=0)
    curve_selection_points['real_y'] = ((curve_selection_points['y'] - y_min[0])/(y_diff[0]))*(y_diff[1]) + y_min[1]
    
    if (curve_name.value == '' or set_name.value == '') and ALERTS_ON:
        
        alert_message('No curve set / curve name specified.', 'Please enter a curve set name and curve name.')
        return
        
    if (set_name.value in list(pushed_curve_sets_and_names.keys())):
        with console_output:
            clear_output()
            print('Checking previous digitizations...')
        if (curve_name.value in pushed_curve_sets_and_names[set_name.value]) and ALERTS_ON:
            alert_message('Curve set / curve name has already been pushed to Seeq.', 'Please specify a new curve set/name and try again.')
            return
        
    with console_output:
        clear_output()
        print('Retrieving asset...')
    asset = pdz.get_asset(asset_name_to_item_id_dict[asset_name.value], trees_api)
    with console_output:
        clear_output()
        print('Updating pltdgtz property...')
    pdz.update_pltdgtz_property(asset, curve_selection_points, set_name.value, curve_name.value, items_api)

    push_to_seeq.disabled = True

    try:
        pushed_curve_sets_and_names[set_name.value].append(curve_name.value)
    except KeyError:
        pushed_curve_sets_and_names.update({set_name.value:[curve_name.value]})

    with console_output:
        clear_output()
        print('Pushing formula...')

    formula_push_results = pdz.create_and_push_formula(
        curve_set=set_name.value, curve_name=curve_name.value, 
        asset_id=asset.id, 
        auth_token=spy.client.auth_token, 
        x_axis_id=x_axis_id
    )

    with console_output:
        clear_output()
        print('Updating workstep...')

    pdz.modify_workstep(workbook=workbook, worksheet=worksheet, 
                    formula_push_results=formula_push_results, trees_api=trees_api, workbooks_api=workbooks_api,
                    x_range=(min(curve_selection_points['real_x']), max(curve_selection_points['real_x'])), 
                    y_range=(min(curve_selection_points['real_y']), max(curve_selection_points['real_y']))
                   )

    with console_output:
        clear_output()
        print('Done. You may digitize another curve, or, if done, close this window.')

    digitize_another_button.disabled=False
            
    return

def on_asset_selection(change):
    if change['type'] == 'change' and change['name'] == 'value':
        set_name.value = ''
        curve_name.value = ''
        
        global curve_set_dict, existing_set_names
        
        with console_output:
            clear_output()
            print('Searching for existing curve sets...')
        asset = pdz.get_asset(asset_name_to_item_id_dict[change['new']], trees_api)
        pltdgtz_prop = pdz.get_pltdgtz_property(asset.id, items_api)
        if pltdgtz_prop is None:
            curve_set_dict = {}
        else:
            curve_set_dict = json.loads(pltdgtz_prop.value)
        with console_output:
            print('Found {} existing curve sets... Select from dropdown, or type new.'.format(len(curve_set_dict)))
        existing_set_names = get_existing_set_names(curve_set_dict)
        set_name.options = existing_set_names
        

def on_curveset_selection(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with console_output:
            curve_name.options = get_existing_curve_names(curve_set_dict, change['new'])


    
# assign functions
approve_axis_point.on_click(on_axis_point_approval)
file_upload.observe(on_file_upload, names='value')
clear_selection.on_click(on_clear_selection)
push_to_seeq.on_click(on_push_to_seeq)
digitize_another_button.on_click(on_digitize_another)
asset_name.observe(on_asset_selection)
set_name.observe(on_curveset_selection)



In [12]:
# App Layout
header = HTML("""<div align="left"><h1>Plot Digitizer</h1>
</div><h5>Begin by uploading an image you wish to digitize</h5>""")

upload_and_start = HBox((file_upload,))

active_area = HBox((fig_output, user_input), layout=Layout(justify_content='space-between'))

vbox_main = VBox((header,upload_and_start,active_area,console_output))
vbox_main.layout = Layout(width='100%', min_height='500px')

In [13]:
ALERTS_ON = True
display(vbox_main)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>